## Helpers

In [8]:
# Used to remove the dashes while verifying a key.
def removeChar(inputWChar, posByte):
    posWChar = posByte >> 1
    return inputWChar[:posWChar] + inputWChar[posWChar+1:]


# Split a given DWORD into bytes for checksum calculation.
def dwordToChars(value):
    return [
        (value >> 0) & 0xff,
        (value >> 8) & 0xff,
        (value >> 16) & 0xff,
        (value >> 24) & 0xff,
    ]

## Checksum

In [9]:
#
# Updates the checksum with the given bytes.
#
def updateChecksum(oldChecksum, inputBytes):
    newChecksum = oldChecksum
    for byte in inputBytes:
        newChecksum ^= byte << 24
        for _ in range(0, 8):
            if newChecksum & 0x80000000 == 0:
                newChecksum = newChecksum << 1
            else:
                newChecksum = (newChecksum << 1) ^ 0x4C11DB7
    return newChecksum & 0xffffffff

## Guess key filler

In [10]:
def guessKeyFiller(subKey1, info3, info4):
    results = []
    seed = updateChecksum(-1, dwordToChars(info3) + dwordToChars(info4))
    info1 = subKey1 ^ (subKey1 << 7)
    minDiff = 0xffffffff
    for guess in range(0, 1 << 25):
        check = updateChecksum(seed, dwordToChars(guess)) & 0x1ffffff
        diff = abs(check - subKey1)
        if diff < minDiff:
            print(f"diff = {diff} on guess = {guess:0>8x}")
            minDiff = diff
        if subKey1 == check:
            print(f"match for guess = {guess:0>8x} subKey1 = {subKey1:0>8x}")
            results += [guess]

    return results

## Process key data

In [11]:
# convert a block of characters into an DWORD
#
# - 5 bits per character
# - chars are ordered "little endian":
#   - first char provides the lowest 5 bits
#   - last char provides highest 5 bits
#
def decodeSubKey(key, start, length):
    result = 0
    while length > 0:
        length -= 1
        currentChar = key[start + length]
        match currentChar:
            case 'W':
                # obfuscating pattern: => 0x00 (like '0')
                offset = 0
            case 'X':
                # obfuscating pattern: => 0x18
                offset = 0x18
            case 'Y':
                # obfuscating pattern: => 0x01 (like '1')
                offset = 1
            case 'Z':
                # obfuscating pattern: => 0x12
                offset = 0x12
            case _ if ord(currentChar) - ord('0') <= 9:
                # numbers 0...9 => bit patterns 0x00...0x09
                offset = ord(currentChar) - ord('0')
            case _:
                # letters A...V => bit patterns 0x0a...0x1f
                offset = ord(currentChar) - ord('7')
        result = (result << 5) + offset
    return result & 0xffffffff


# decode general key layout:
#
# 1 = 25bit: (de-)obfuscator key and expected checksum
# 2 = 25bit: independent block to adjust actual checksum
# 3 = 32bit: obfuscated product and version info
# 4 = 32bit: obfuscated license expiration info
#
# - key consists originally of 5x5=25 chars
# - the logic below extracts only 2x7+2x5=24 chars
# - the therefore unused char at position 14 is copied to position 2 before splitting the key
# - part 3 and 4 are swapped, so the key layout before decoding is:
#
# 33_33-33444-44443-22222-11111
#   ^             |
#   |             |
#   +-------------+

def decodeKey(key):
    if len(key) == 29:
        # remove dashes
        for seperatorPos in [46, 34, 22, 10]:
            key = removeChar(key, seperatorPos)

        # obfuscation in 3rd char of part 3
        key = key[:2] + key[14] + key[3:]

        # split parts
        part1 = decodeSubKey(key, 20, 5)
        part2 = decodeSubKey(key, 15, 5)
        part3 = decodeSubKey(key, 0, 7)
        part4 = decodeSubKey(key, 7, 7)

        # deobfuscate part3 and part4
        info1 = part1 ^ (part1 << 7)
        info2 = part2
        info3 = info1 ^ part3 ^ 0x12345678
        info4 = info1 ^ part4 ^ 0x87654321

        return (part1, info1, info2, info3, info4)

    else:
        print("Incorrect key length")
        return (0, 0, 0, 0, 0)

# extract product and version data from info3 and info4 (deobfuscated)


def decodeKeyData(key):
    (_, _, _, info3, info4) = decodeKey(key)

    t0 = info3 >> 21           # 11111111 11100000 00000000 00000000
    t1 = (info3 >> 16) & 0x1f  # 00000000 00011111 00000000 00000000
    version = info3 & 0xffff
    t2 = version >> 5          # 00000000 00000000 11111111 11100000
    t3 = version & 0x1f        # 00000000 00000000 00000000 00011111

    # ffffffff ffffffff 00000000 00000000
    totalMonth = (info4 >> 16) & 0xffff
    if totalMonth > 0:
        t4 = 2000 + int(totalMonth / 12)  # year
        t5 = totalMonth % 12  # month
    else:
        t4 = 0
        t5 = 0
    # 00000000 00000000 ffffffff ffffffff
    t6 = info4 & 0xffff

    print(f"product 1: {t0:>8d} expected = 3")
    print(f"product 2: {t1:>8d} expected = 0")
    print(f"version 1: {t2:>8d} expected >= 100")
    print(f"version 2: {t3:>8d} ???")
    print(f"year:      {t4:>8d} expected = 0")
    print(f"month:     {t5:>8d} expected = 0")
    print(f"???        {t6:>8d} ???")


# reverse of decodeSubKey()
# does not use the characters 'W'-'Z'
def encodeSubKey(value, length):
    subKey = ""
    while length > 0:
        bits = value & 0x1f
        if bits <= 9:
            subKey += chr(0x30+bits)
        else:
            subKey += chr(0x37+bits)
        value >>= 5
        length -= 1
    return subKey


# create a key for a given product, version and expiration
def createKeys(prod1, prod2, v1, v2, year, month, unknown):
    # iterate over the expected checksum subkey (2**25 possibilities, ~32M)
    for subKey1 in range(0, 1 << 25):
        print(f"trying subKey1 = {subKey1}")

        # create obfuscator key
        info1 = subKey1 ^ (subKey1 << 7)

        # encode expiration data
        info4 = (unknown & 0xffff)
        if year > 0:
            info4 |= (((year - 2000) * 12 + month) & 0xffff) << 16

        # encode product and version data
        info3 = (((prod1 & 0x7ff) << 21) |
                 ((prod2 & 0x1f) << 16) |
                 ((v1 & 0x7ff) << 5) |
                 (v2 & 0x1f))

        # part 1, 2 and 4 are now known => generate key strings
        part1 = encodeSubKey(info3 ^ info1 ^ 0x12345678, 7)
        part2 = encodeSubKey(info4 ^ info1 ^ 0x87654321, 7)
        part4 = encodeSubKey(subKey1, 5)

        # search valid adjustment subkeys (2**25 possibilities, ~32M)
        for filler in guessKeyFiller(subKey1, info3, info4):
            part3 = encodeSubKey(filler, 5)

            # create key with 5-char-blocks and dashes
            key = (part1[:5] + "-" +
                   part1[5:7] + part2[:3] + "-" +
                   part2[3:7] + part1[2] + "-" +
                   part3 + "-" +
                   part4)

            print(f"key = {key}")

## Top level validation

In [12]:
def verifyKey(key):
    (expectedChecksum, _, info2, info3, info4) = decodeKey(key)

    info = dwordToChars(info3) + dwordToChars(info4) + dwordToChars(info2)
    actualChecksum = updateChecksum(-1, info) & 0x1ffffff

    decodeKeyData(key)
    if expectedChecksum == actualChecksum:
        return 1
    else:
        print("Incorrect key")
        return 0

## Test key

In [13]:
key = "00000-00000-00000-00000-00000"

print(f"{verifyKey(key)}")

product 1:      145 expected = 3
product 2:       20 expected = 0
version 1:      691 expected >= 100
version 2:       24 ???
year:          4888 expected = 0
month:            5 expected = 0
???           17185 ???
Incorrect key
0


## Create Key

In [14]:
# createKeys(3, 0, 100, 0, 0, 0, 0)